
# Optimizing the Schedule of a Triple Major Student at UW-Madison #

#### Agastya Vinchhi , Haoyang Ding, Lauren Haefer

*****

<!-- TOC -->
<a id="toc-top"></a>

# Table of Contents

- [1. Introduction](#sec-1)
- [2. Model Definition](#sec-2)
- [3. Data and Implimentation](#sec-3)
- [4. Model - all cases](#sec-4)
  - [4.0 Base Case](#sec-4-0)
  - [4.1 Case I - Weighted Objective for CS Courses](#sec-4-1)
  - [4.2 Case II - Focused Curriculum](#sec-4-2)
  - [4.3 Case III - Late Major Addition](#sec-4-3)
- [5. Limitations, Future Work, and Conclusion](#sec-5)



<!-- 1 -->
<a id="sec-1"></a>
## 1. Introduction
[↑ Back to top](#toc-top)

Pursuing a triple major in Data Science, Economics, and Computer Science at the University of Wisconsin-Madison is an ambitious goal that offers a powerful combination of skills. These fields complement each other well with data analysis, economic modeling, and programming all playing a big role in the modern world. However, figuring out how to fit the core requirements for three majors into one manageable schedule is no easy task. Furthermore, our project aims to tackle this challenge by creating an optimization model that helps students map out the most efficient class schedule, utilizing overlapping courses between the three majors, to save time and reduce stress.

Together, these majors open up opportunities in areas like machine learning, financial modeling, and software development. However, the workload can seem overwhelming, and the logistics of balancing core classes, electives, prerequisites, and credit limits can feel like a puzzle. That’s where this project comes in—to take some of the guesswork out of the process and help students see a clear path forward. Our model uses linear programming, to find the best way to meet the requirements for all three majors while minimizing the time it takes to graduate. The model will take into account things like which courses have prerequisites, and how many credits a student can realistically take in a semester. We will also focus on finding overlapping courses that can count toward multiple majors. For example, many Data Science electives also count as Computer Science electives, this way, a student does not have to repeat courses and can comfertabley satisfy requiremnts.

In our project we provide four different models:
1) A base optimization model which will print out an optimal schedule to satisfy the core requiremtns for the three majors
2) Three case studies of customized optimization models to provide different situations a student might be in when trying to tripple major (take specific courses, optimality trade off - focusing more on a specific major, taking specific courses later, etc.)

The goal is to create a practical tool that students and advisors can use to make smarter decisions about course planning. Instead of spending hours piecing together schedules, students can use this model to get a personalized plan that balances efficiency and flexibility. Plus, the approach could work for other combinations of majors in the future.

The data used in this model was hand-collected from the individual major pages offered through UW-Madison's pulic website. Course name, credits, and difficulty were put into a CSV file and loaded into our Jupyter Notebook. We organized the data, cleaned it, and parsed it for further analysis.

<!-- 2 -->
<a id="sec-2"></a>
## 2. Model Definition
[↑ Back to top](#toc-top)

Base Model: 
Bellow, we have formed a mathematical optimization model to achieve our desired output. This is a Mixed Interger Programing model since we define binary vairables for the courses we take for all x_i.

Note: It is important to note that each major has its own core and elective requirements out of many different course options. Hence, each constraint in our model will represent a course that has to be taken for that specific requirement. Moreover, we can have elective courses that can be taken from the elective requirement or from core requirements - this complication has been modeled bellow.

$$
\begin{aligned}
\underset{x, e}{\text{minimize}}\qquad & c^\top x \\
\text{subject to:} \qquad & A x + e \geq b \\
& x \in \{0, 1\}, \quad e \geq 0, \quad b \geq 0
\end{aligned}
$$

1) Decision Variable: 
    - Let x ∈ {0,1}, which is a Binary Variable whetjher each course is taken or not; here x_i indicate whether a course is taken (x_i = 1) or not (x_i = 0). 

2) Objective Function: 
    - Let c' be a vector of ones and let x_i be the decision variable we aim to minimize. It is clear that here, we minimize the total number of courses taken while satisfying core, elective, and credit requirements.

3) Constraints: 
    - Let A be a matrix where each row represents a constraint equation's coefficients for courses. Consequently, we have the vector b representing the total number of required courses for each constraints. Hence, this will help us formulate each constraint, where out of all the different options of courses from matrix A, we must take exactly b number of courses to satisfy the major requiremnt. Notice the greater than equal to sign, ensuring that we are always taking atleast a total of b number of courses.

    - Finally, we define e >= 0, which is an auxilary slack variable for elective constraints. Some majors have electives, where you can take a course from the elective requiremnt or from a core requirement. In such cases, we utilize an external variable e to either remove or add the total number of courses required from the requiremnt constraint (to understand this more, view the comments in the base model). The introduction of this variable will disallow the model to "double dip" elective courses, furthermore, producing incorrect outputs.

    - For instance, let us say for some constraint j, we need 2 courses from a range of Computer Science courses. Here, our A matrix will represent the different number of courses we can choose from. The vector b would be = 2, as we need 2 courses to satisfy the requirement. And let us say that we choose to take a course to satisfy for some constraint k, then we can set e = -1, allowing us to then take 2 + 1 = 3 courses for that requirement instead of 2. 



<!-- 3 -->
<a id="sec-3"></a>
## 3. Data and Implimentation
[↑ Back to top](#toc-top)

First, we define parse our data and define some helper functions before we proceed with the modeling.

In [15]:
#Here, we are exctracting and parsing all our data from our CSV file which containts all the courses of each major. 
#Obtaining a range of different courses that are possible to take, allows the model to optimize correctly.

using CSV
using DataFrames

df = CSV.read("uwcourses_updated.csv", DataFrame)

list_courses = []
for col in eachcol(df)
    for course in col
        if ismissing(course) == true
            continue
        end
        push!(list_courses, course)
    end
end
courses = collect(Set(list_courses)) 

println(df)

91×3 DataFrame
 Row │ computer-science               data-science                   economics                         
     │ String31                       String31?                      String?                           
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ COMP SCI/MATH 240              MATH 221                       MATH 221
   2 │ COMP SCI/E C E 252             MATH 222                       MATH 171
   3 │ COMP SCI 300                   STAT 240                       MATH 217
   4 │ COMP SCI/E C E 354             STAT 340                       MATH 211
   5 │ COMP SCI 400                   COMP SCI 220                   MATH 213
   6 │ MATH 221                       COMP SCI 300                   MATH 211
   7 │ MATH 222                       COMP SCI 320                   ECON 205
   8 │ MATH 171                       L I S 461                      ECON 310
   9 │ MATH 217                       MATH 217   

In [16]:
#In the future, we want to look at the difficulty of the course, allowing us to further investigate and optimize accordingly.
#Hence, we extract the course numbers of all the courses

courses_df = CSV.read("uwcourses_updated.csv", DataFrame)

# Convert all columns to same type 
for col in names(courses_df)
    courses_df[!, col] = convert(Vector{String}, string.(courses_df[!, col]))
end

# Function to clean the input and extract the numerical part of the course name
function extract_course_number(course_name)
    if typeof(course_name) == String
        cleaned_course_name = strip(course_name)
        match_obj = match(r"\d+", cleaned_course_name)
        return match_obj !== nothing ? parse(Int, match_obj.match) : missing
    else
        return missing
    end
end

course_numbers_df = DataFrame()

# Iterate over each column and extract the course numbers
for col in names(courses_df)
    extracted_numbers = [extract_course_number(course_name) for course_name in courses_df[!, col]]
    course_numbers_df[!, col] = extracted_numbers
end

println(course_numbers_df)

91×3 DataFrame
 Row │ computer-science  data-science  economics 
     │ Int64             Int64?        Int64?    
─────┼───────────────────────────────────────────
   1 │              240           221        221
   2 │              252           222        171
   3 │              300           240        217
   4 │              354           340        211
   5 │              400           220        213
   6 │              221           300        211
   7 │              222           320        205
   8 │              171           461        310
   9 │              217           217        400
  10 │              222           570        410
  11 │              320           532        309
  12 │              340           539        431
  13 │              341           540        311
  14 │              375           656        324
  15 │              309           521        340
  16 │              311           535        101
  17 │              324           361        102
  

In [17]:
#Here, we are creating a dictionary called difficulty, to extract how hard a course is based on the course number. 
#This will help us later on when we aim to put more weight on difficulty of a course.

difficulty = Dict()


for i in 1:nrow(courses_df)
    for col in names(courses_df)
        course_name = courses_df[i, col]
        difficulty_score = course_numbers_df[i, col]
        if !ismissing(course_name) && !ismissing(difficulty_score)
            difficulty[string(course_name)] = difficulty_score
        end
    end
end

println(difficulty)

Dict{Any, Any}("ECON 522" => 522, "GEOG 574" => 574, "STAT 433" => 433, "COMP SCI/MATH 240" => 240, "SOIL SCI 585" => 585, "ECON 409" => 409, "STAT 312" => 312, "ECON 570" => 570, "COMP SCI 412" => 412, "COMP SCI/E C E 252" => 252, "ECON/FINANCE 320" => 320, "ECON 468" => 468, "COMP SCI/B M I 576" => 576, "COMP SCI 541" => 541, "ECON 455" => 455, "GEOG 378" => 378, "GEN BUS 656" => 656, "ECON/HIST SCI 305" => 305, "ECON/AAE 371" => 371, "MATH 171" => 171, "L I S 501" => 501, "COMP SCI/E C E/ISYE 524" => 524, "COMP SCI/E C E 552" => 552, "COMP SCI/DS 579" => 579, "ECON 461" => 461, "MATH/STAT 309" => 309, "ECON/AAE/ENVIR ST 343" => 343, "ECON 460" => 460, "ECON 311" => 311, "STAT 453" => 453, "ECON 442" => 442, "STAT 240" => 240, "L I S 464" => 464, "MATH 535" => 535, "ECON 111" => 111, "COMP SCI/E C E 506" => 506, "I SY E 412" => 412, "COMP SCI 566" => 566, "L I S 407" => 407, "ECON/REAL EST/URB R PL 420" => 420, "COMP SCI/I SY E/M E 558" => 558, "ECON 302" => 302, "COMP SCI 220" => 22

Another CSV file was utilized to match each course with its respective credit hours. This will ensure the student takes enough classes to graduate while not overloading any semester with too many credits.

In [18]:
credits_df = CSV.read("CS524-optimization-courses-credit-courses.csv", DataFrame)
credits = Dict()

for i in 1:nrow(courses_df)
    for col in names(courses_df)
        course_name = courses_df[i, col]
        credit = credits_df[i, col]
        if !ismissing(course_name) && !ismissing(credits_df)
            credits[string(course_name)] = credit
        end
    end
end

credits

Dict{Any, Any} with 190 entries:
  "ECON 522"           => 4
  "GEOG 574"           => 4
  "STAT 433"           => 3
  "COMP SCI/MATH 240"  => 3
  "SOIL SCI 585"       => 3
  "ECON 409"           => 4
  "STAT 312"           => 3
  "ECON 570"           => 4
  "COMP SCI 412"       => 3
  "COMP SCI/E C E 252" => 3
  "ECON/FINANCE 320"   => 3
  "ECON 468"           => 4
  "COMP SCI/B M I 576" => 3
  "COMP SCI 541"       => 3
  "ECON 455"           => 4
  "GEOG 378"           => 4
  "GEN BUS 656"        => 3
  "ECON/HIST SCI 305"  => 4
  "ECON/AAE 371"       => 3
  ⋮                    => ⋮

New function created to group courses based off their respective major. 

In [19]:
#This function group courses based on the major
function group_courses(course_list, difficulty)
    grouped_courses = Dict{String, Vector{Int}}()

    # Iterate over each course in the course list
    for course in course_list
        for (course_name, course_number) in difficulty
            # Check if the course name matches
            if course == course_name
                # Extract the major from the course name (everything before the last space)
                parts = split(course_name)
                major = join(parts[1:end-1], " ")

                # Add the course number to the appropriate major in the dictionary
                if haskey(grouped_courses, major)
                    push!(grouped_courses[major], course_number)
                else
                    grouped_courses[major] = [course_number]
                end
            end
        end
    end
    
    # Sort the course numbers for each major for easier readability
    for (major, numbers) in grouped_courses
        grouped_courses[major] = sort(numbers)
    end

    return grouped_courses
end

#This functions helps us print out a schedule, based on the optimal soltuion found. We use this function after every model is optimized.
function create_schedule(courses::Dict{String, Vector{Int}})
    # Flatten the input dictionary into a list of (subject, course) tuples
    sorted_courses = []
    for (subject, course_list) in courses
        append!(sorted_courses, [(subject, course) for course in course_list])
    end
    
    # Sort the courses by their level 
    sorted_courses = sort(sorted_courses, by = x -> parse(Int, string(x[2])[1]), rev=false)
    
    # Dict to store the schedule for 8 semesters
    schedule = Dict("Semester $i" => [] for i in 1:8)
    
    # Dictionary to track the last semester a course from each subject was taken
    last_taken = Dict{String, Int}()
    
    # List to keep track of unadded courses
    unadded_courses = []

    # Initial distribution of courses across the semesters
    for (subject, course) in sorted_courses
        added = false
        for semester in 1:8
            if length(schedule["Semester $semester"]) < 4 && !(subject in [x[1] for x in schedule["Semester $semester"]])
                push!(schedule["Semester $semester"], (subject, course))
                last_taken[subject] = semester
                added = true
                break
            end
        end
        
        # Track courses that were not added in the first round
        if !added
            push!(unadded_courses, (subject, course))
        end
    end

    # Attempt to place unadded courses in any available semester slots
    for (subject, course) in unadded_courses
        for semester in 1:8
            if length(schedule["Semester $semester"]) < 4
                push!(schedule["Semester $semester"], (subject, course))
                break
            end
        end
    end

    return schedule
end

create_schedule (generic function with 1 method)

<!-- 4 (parent section for all model cases) -->
<a id="sec-4"></a>
## 4. MIP Model — all cases
[↑ Back to top](#toc-top)

For our project, we created 4 scenarios. The first is a general base case solving our main objective. The next 3 case studies are aimed at increasing
the realness and our overall understanding of our model by introducting 3 different scenarious of students who could benefit from using our mode.

Note: The output of each of these models have been printed out after each cell in a table format, where we collate a schedule of all the optimal soluation i.e, the number of courses requried to be taken per semester.

<!-- 4.0 -->
<a id="sec-4-0"></a>
### 4.0 Base Case

Below is the base case of our model. The objective is to minimize the difficulty of courses taken while meeting all requriements. This difficulty is based off of the first number of the course name (524 would be considered more difficult than a 400 level course). In order to keep track of courses, we introduced a binary variable, course_taken, that has a value of 1 if a course should be taken, and 0 if it should not be. The constraints are based off university requirements for all 3 majors. 

All of our constraints are based off the different requriements across all 3 majors. There are some courses, such as Calculus, that are required by all 3 majors. But all other constraints can be seen categorized by the major they are fulfulling. The binary variable is seen in conjuction with each individual course that can fulfill that requirement and set to the number of courses required in that specific category. One important note on our model is that we only considered 'core' classes. There are several liberal arts requirements, however, due to the fact that there are 100's if not 1000's of electives to choose from that will not impact the major courses themselves, we felt it was better to leave these our to not create further noise and distraction in our ourcomes. But, these have been accounted for, in our constraints and schedule output we account for these classes with credit limits and the number of core classes taken per semester.

In the early stages of our modeling we were having difficulty getting our constraints to work correctly, mainly the elective requirements for all majors. Each major requires students to take certain categories of classes AND a certain number of additional electives, which can be additional classes from these categories. For example, Computer Science students are required to take 1 course in the applications category. In addition to this they must take 2 electives, if a student particularly liked applications, they can choose their elective courses to be applications. Due to this, our optimal solution was allowing elective courses to simultaneously count for required categories. This was lowering the number of courses a student must take, and taking away from the integrity and legitimacy of our model. In order to combat this issue, we introduced 7 variables to represent the different electives. These variables were initialized to 0. Throughout our model, if a course in a particular area is taken the respective variables value increases by one. At the end, the counts of these variables are subtracted from each major constraint to ensure that no courses are being counted for different requriements and allowing for a more accurate model.

In [20]:
using JuMP, GLPK

# Define the model with the GLPK solver
model = Model(GLPK.Optimizer)

# Create a binary variable for each course
@variable(model, course_taken[courses], Bin)

#All elective variables
@variable(model, e_cs_theory >= 0)
@variable(model, e_cs_software_hardware >= 0)
@variable(model, e_cs_applications >= 0)
@variable(model, e_ds_machine_learning >= 0)
@variable(model, e_ds_advanced_computing >= 0)
@variable(model, e_ds_statistical_modelling >= 0)
@variable(model, e_econ_two_courses >= 0)

#Computer Science Constraints
@constraint(
    model,
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI/MATH 240"] + 
    course_taken["COMP SCI/E C E 252"] + 
    course_taken["COMP SCI/E C E 354"] + 
    course_taken["COMP SCI 200"] + 
    course_taken["COMP SCI 400"] >= 6 
)

@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)

@constraint(
    model,
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["MATH 331"] + 
    course_taken["STAT 333"] + 
    course_taken["STAT 340"] + 
    course_taken["STAT 371"] + 
    course_taken["STAT/MATH 431"] + 
    course_taken["MATH 531"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI 577"] + 
    course_taken["COMP SCI 520"] >= 1 + e_cs_theory
)
@constraint(
    model,
    course_taken["COMP SCI 407"] + 
    course_taken["COMP SCI/E C E 506"] + 
    course_taken["COMP SCI 536"] + 
    course_taken["COMP SCI 538"] + 
    course_taken["COMP SCI 537"] + 
    course_taken["COMP SCI 542"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI/E C E 552"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 640"] + 
    course_taken["COMP SCI 642"] >= 2 + e_cs_software_hardware
)
@constraint(
    model,
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI 534"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["COMP SCI 541"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI 566"] + 
    course_taken["COMP SCI 570"] + 
    course_taken["COMP SCI 571"] >= 1 + e_cs_applications
)
@constraint(
    model,
    course_taken["COMP SCI/E C E/MATH 435"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH/STAT 475"] + 
    course_taken["COMP SCI/DS/I SY E 518"] + 
    course_taken["COMP SCI/I SY E 526"] + 
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI/E C E/M E 539"] +  
    course_taken["COMP SCI/I SY E/M E 558"] + 
    course_taken["COMP SCI/E C E 561"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["COMP SCI/DS 579"] + 
    course_taken["COMP SCI 620"] + 
    course_taken["COMP SCI/I SY E 635"] + 
    course_taken["COMP SCI 639"] >= 2 - e_cs_theory - e_cs_software_hardware - e_cs_applications
)

#Data Science Constraints
@constraint(
    model,
    course_taken["MATH 221"] + course_taken["MATH 217"] >= 1
)

@constraint(
    model,
    course_taken["MATH 222"] >= 1
)
@constraint(
    model,
    course_taken["STAT 240"] +
    course_taken["STAT 340"] +
    course_taken["COMP SCI 220"] + 
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI 320"] +
    course_taken["L I S 461"] + 
    course_taken["E C E/I SY E 570"] >= 7
)
@constraint(
    model,
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E/M E 539"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["GEN BUS 656"] + 
    course_taken["I SY E 521"] + 
    course_taken["MATH 535"] + 
    course_taken["PHYSICS 361"] + 
    course_taken["STAT 451"] + 
    course_taken["STAT 453"] >= 1 + e_ds_machine_learning
)
@constraint(
    model,
    course_taken["COMP SCI 400"] + 
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["GEOG 573"] + 
    course_taken["GEOG 574"] + 
    course_taken["MATH 444"] >= 1 + e_ds_advanced_computing
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 460"] + 
    course_taken["GEOG 579"] + 
    course_taken["I SY E 575"] + 
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT/MATH 310"] + 
    course_taken["STAT 312"] + 
    course_taken["STAT 349"] + 
    course_taken["STAT 351"] + 
    course_taken["STAT 421"] + 
    course_taken["STAT/M E 424"] + 
    course_taken["STAT 436"] + 
    course_taken["STAT 443"] + 
    course_taken["STAT 456"] + 
    course_taken["STAT 461"] + 
    course_taken["STAT 575"] + 
    course_taken["MATH 531"] + 
    course_taken["MATH/I SY E/OTM/STAT 632"] + 
    course_taken["MATH 635"] >= 1 + e_ds_statistical_modelling
)
@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI 577"] + 
    course_taken["E C E 203"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 695"] + 
    course_taken["GEOG 378"] + 
    course_taken["GEOG 572"] + 
    course_taken["GEOG 575"] + 
    course_taken["I SY E 323"] + 
    course_taken["I SY E 412"] + 
    course_taken["I SY E/M E 512"] + 
    course_taken["I SY E 612"] + 
    course_taken["INFO SYS 322"] + 
    course_taken["L I S 407"] + 
    course_taken["L I S 440"] + 
    course_taken["L I S 464"] + 
    course_taken["L I S 501"] + 
    course_taken["LSC 460"] + 
    course_taken["LSC 660"] + 
    course_taken["MATH 331"] + 
    course_taken["SOC 351"] + 
    course_taken["SOC/C&E SOC 618"] + 
    course_taken["SOC/C&E SOC 693"] + 
    course_taken["SOIL SCI 585"] + 
    course_taken["STAT 405"] + 
    course_taken["STAT 433"] >= 2 - e_ds_machine_learning - e_ds_advanced_computing - e_ds_statistical_modelling
)

#Econ Constraints
@constraint(
    model,
    2*course_taken["MATH 221"] + 
    course_taken["MATH 211"] + 
    course_taken["ECON 205"] >= 2
)
@constraint(
    model,
    course_taken["ECON 310"] + 
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["MATH/STAT 309"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["STAT 340"] >= 1
)
@constraint(
    model,
    course_taken["ECON 101"] + 
    course_taken["ECON 102"] + 
    2*course_taken["ECON 111"] >= 2
)
@constraint(
    model,
    course_taken["ECON 301"] + course_taken["ECON 302"]  >= 2
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 409"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 435"] + 
    course_taken["ECON 441"] + 
    course_taken["ECON 442"] + 
    course_taken["ECON 448"] + 
    course_taken["ECON 450"] + 
    course_taken["ECON 451"] + 
    course_taken["ECON 455"] + 
    course_taken["ECON 458"] + 
    course_taken["ECON 460"] + 
    course_taken["ECON 461"] + 
    course_taken["ECON 464"] + 
    course_taken["ECON 467"] + 
    course_taken["ECON 468"] + 
    course_taken["ECON 475"] + 
    course_taken["ECON/FINANCE 503"] + 
    course_taken["ECON 521"] + 
    course_taken["ECON 522"] + 
    course_taken["ECON/RMI 530"] + 
    course_taken["ECON/POP HLTH/PUB AFFR 548"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 580"] + 
    course_taken["ECON 621"] + 
    course_taken["ECON 623"] + 
    course_taken["ECON 661"] + 
    course_taken["ECON 664"] + 
    course_taken["ECON 666"] + 
    course_taken["ECON 690"] + 
    course_taken["ECON 695"] >= 2 + e_econ_two_courses
)
@constraint(
    model,
    course_taken["ECON/FINANCE 300"] + 
    course_taken["ECON/HIST SCI 305"] + 
    course_taken["ECON/AAE/REAL EST/URB R PL 306"] + 
    course_taken["ECON 309"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON/FINANCE 320"] + 
    course_taken["ECON 321"] + 
    course_taken["ECON 330"] + 
    course_taken["ECON/AAE/ENVIR ST 343"] + 
    course_taken["ECON 355"] + 
    course_taken["ECON 370"] + 
    course_taken["ECON/AAE 371"] + 
    course_taken["ECON 390"] + 
    course_taken["ECON/REAL EST/URB R PL 420"] + 
    course_taken["ECON/AAE 421"] + 
    course_taken["ECON/ENVIR ST/POLI SCI/URB R PL 449"] + 
    course_taken["ECON/AAE/INTL BUS 462"] + 
    course_taken["ECON 465"] + 
    course_taken["ECON/HISTORY 466"] + 
    course_taken["ECON/AAE 473"] + 
    course_taken["ECON/AAE 474"] + 
    course_taken["ECON/AAE 477"] + 
    course_taken["ECON/PHILOS 524"] + 
    course_taken["ECON/AAE 526"] + 
    course_taken["ECON/AAE/F&W ECOL 531"] + 
    course_taken["ECON/SOC 663"] + 
    course_taken["ECON/AAE/ENVIR ST/URB R PL 671"] >= 2 - e_econ_two_courses
)

@constraint(
    model,
    sum(credits[course] * course_taken[course] for course in courses) >= 48
)

@objective(model, Min, sum(course_taken[course] for course in courses))

# Solve the optimization problem
optimize!(model)

# Check if a solution was found
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found. Courses to enroll with lowest number of courses:")
    final_taken = []
    for course in courses
        if value(course_taken[course]) > 0.5
            push!(final_taken, course)
            println(course)
        end
    end
else
    println("No optimal solution found.")
end

Optimal solution found. Courses to enroll with lowest number of courses:
ECON 400
COMP SCI/E C E 354
COMP SCI/MATH 240
COMP SCI 537
ECON 570
ECON 301
COMP SCI 412
COMP SCI/E C E 252
ECON 321
COMP SCI/DS/I SY E 518
COMP SCI/E C E/M E 532
STAT 240
MATH 222
ECON 111
ECON 315
ECON 302
COMP SCI 520
COMP SCI 220
E C E/I SY E 570
MATH 375
COMP SCI 564
COMP SCI 200
COMP SCI 320
COMP SCI 300
L I S 461
COMP SCI 400
MATH 221
STAT 340


In [21]:
#Print Schedule Result
result = group_courses(final_taken, difficulty)
schedule = create_schedule(result)
for i in 1:8
    semester_key = "Semester $i"
    if length(schedule[semester_key]) > 0
        println("$semester_key: ", join(["$course" for course in schedule[semester_key]], ", "))
    else
        println("$semester_key: No courses scheduled")
    end
end

Semester 1: ("ECON", 111), ("MATH", 221), ("COMP SCI/E C E", 252), ("COMP SCI", 200)
Semester 2: ("MATH", 222), ("COMP SCI", 220), ("STAT", 240), ("COMP SCI/MATH", 240)
Semester 3: ("MATH", 375), ("ECON", 301), ("COMP SCI/E C E", 354), ("COMP SCI", 300)
Semester 4: ("ECON", 302), ("COMP SCI", 320), ("STAT", 340), ("L I S", 461)
Semester 5: ("ECON", 315), ("COMP SCI", 400), ("E C E/I SY E", 570), ("COMP SCI/E C E/M E", 532)
Semester 6: ("ECON", 321), ("COMP SCI", 412), ("COMP SCI/DS/I SY E", 518), ("COMP SCI", 564)
Semester 7: ("ECON", 400), ("COMP SCI", 520)
Semester 8: ("ECON", 570), ("COMP SCI", 537)


<!-- 4.1 -->
<a id="sec-4-1"></a>
### 4.1 Case I — Weighted Objective for CS Courses
[↑ Back to top](#toc-top)

Situation A: Optimality Trade Off - Looking at the schedule of a student who wants to take the minimum number of courses to triple major, but wants to emphasize on taking more Computer Science courses and difficult courses.

For the next iteration of our model, we adjusted the objective function to place a greater emphasis on CS courses. This update was a valuable step as it increased the complexity of the model while reflecting a realistic scenario: a student who wants to pursue a triple major but has a strong preference for one subject and wants to take diffcult courses. To implement this change, we:
- introduced a penalty for all non-CS courses in the objective function,
- take into consideration the difficulty of the course, from the difficulty dictionary we defined earlier in our objective function,
- add a constraint to ensure that we take a full course load of courses per semester, maximizing the difficulty as much as possible! (taking around 108 credits worth of core courses)

By adding this penalized weight, our model ensures that the student meets the minimum required courses for Economics and Data Science while allowing them to dedicate as much of their academic schedule as possible to Computer Science courses. This change better reflects the real-world decision-making process of students who prioritize one major over others, ensuring that the model remains realistic and aligned with student preferences while still meeting graduation requirements. This has been effectivley done by implimenting a lambda parameter in the objective function (optimality trade off), allowing us to emphasize on Computer Science courses. To add on, we have taken the difficulty of the course into consideration in our objective function, to maximize taking the hardest courses possible at UW-Madison (all upper level courses from the number 400-600!)

In [22]:
#We use HiGHS to solve Situation A - HiGHS helps us solve our optimization problem a lot faster than JuMP. JuMP can take ~ 10 minutes (several minutes), however, HiGHS is a lot faster.
#If HiGHS is not installed, please comment out the bellow line.
# import Pkg; Pkg.add("HiGHS") 

using JuMP, HiGHS 

# Define the model with the HiGHS solver
model = Model(HiGHS.Optimizer)

# Create a binary variable for each course
@variable(model, course_taken[courses], Bin)

#All elective variables
@variable(model, e_cs_theory >= 0)
@variable(model, e_cs_software_hardware >= 0)
@variable(model, e_cs_applications >= 0)
@variable(model, e_ds_machine_learning >= 0)
@variable(model, e_ds_advanced_computing >= 0)
@variable(model, e_ds_statistical_modelling >= 0)
@variable(model, e_econ_two_courses >= 0)

#Computer Science Constraints
@constraint(
    model,
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI/MATH 240"] + 
    course_taken["COMP SCI/E C E 252"] + 
    course_taken["COMP SCI/E C E 354"] + 
    course_taken["COMP SCI 200"] + 
    course_taken["COMP SCI 400"] >= 6
)

@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)

@constraint(
    model,
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["MATH 331"] + 
    course_taken["STAT 333"] + 
    course_taken["STAT 340"] + 
    course_taken["STAT 371"] + 
    course_taken["STAT/MATH 431"] + 
    course_taken["MATH 531"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI 577"] + 
    course_taken["COMP SCI 520"] >= 1 + e_cs_theory
)
@constraint(
    model,
    course_taken["COMP SCI 407"] + 
    course_taken["COMP SCI/E C E 506"] + 
    course_taken["COMP SCI 536"] + 
    course_taken["COMP SCI 538"] + 
    course_taken["COMP SCI 537"] + 
    course_taken["COMP SCI 542"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI/E C E 552"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 640"] + 
    course_taken["COMP SCI 642"] >= 2 + e_cs_software_hardware
)
@constraint(
    model,
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI 534"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["COMP SCI 541"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI 566"] + 
    course_taken["COMP SCI 570"] + 
    course_taken["COMP SCI 571"] >= 1 + e_cs_applications
)
@constraint(
    model,
    course_taken["COMP SCI/E C E/MATH 435"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH/STAT 475"] + 
    course_taken["COMP SCI/DS/I SY E 518"] + 
    course_taken["COMP SCI/I SY E 526"] + 
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI/E C E/M E 539"] +  
    course_taken["COMP SCI/I SY E/M E 558"] + 
    course_taken["COMP SCI/E C E 561"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["COMP SCI/DS 579"] + 
    course_taken["COMP SCI 620"] + 
    course_taken["COMP SCI/I SY E 635"] + 
    course_taken["COMP SCI 639"] >= 2 - e_cs_theory - e_cs_software_hardware - e_cs_applications
)

#Data Science Constraints
@constraint(
    model,
    course_taken["MATH 221"] + course_taken["MATH 217"] >= 1
)

@constraint(
    model,
    course_taken["MATH 222"] >= 1
)
@constraint(
    model,
    course_taken["STAT 240"] +
    course_taken["STAT 340"] +
    course_taken["COMP SCI 220"] + 
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI 320"] +
    course_taken["L I S 461"] + 
    course_taken["E C E/I SY E 570"] >= 7
)
@constraint(
    model,
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E/M E 539"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["GEN BUS 656"] + 
    course_taken["I SY E 521"] + 
    course_taken["MATH 535"] + 
    course_taken["PHYSICS 361"] + 
    course_taken["STAT 451"] + 
    course_taken["STAT 453"] >= 1 + e_ds_machine_learning
)
@constraint(
    model,
    course_taken["COMP SCI 400"] + 
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["GEOG 573"] + 
    course_taken["GEOG 574"] + 
    course_taken["MATH 444"] >= 1 + e_ds_advanced_computing
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 460"] + 
    course_taken["GEOG 579"] + 
    course_taken["I SY E 575"] + 
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT/MATH 310"] + 
    course_taken["STAT 312"] + 
    course_taken["STAT 349"] + 
    course_taken["STAT 351"] + 
    course_taken["STAT 421"] + 
    course_taken["STAT/M E 424"] + 
    course_taken["STAT 436"] + 
    course_taken["STAT 443"] + 
    course_taken["STAT 456"] + 
    course_taken["STAT 461"] + 
    course_taken["STAT 575"] + 
    course_taken["MATH 531"] + 
    course_taken["MATH/I SY E/OTM/STAT 632"] + 
    course_taken["MATH 635"] >= 1 + e_ds_statistical_modelling
)
@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI 577"] + 
    course_taken["E C E 203"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 695"] + 
    course_taken["GEOG 378"] + 
    course_taken["GEOG 572"] + 
    course_taken["GEOG 575"] + 
    course_taken["I SY E 323"] + 
    course_taken["I SY E 412"] + 
    course_taken["I SY E/M E 512"] + 
    course_taken["I SY E 612"] + 
    course_taken["INFO SYS 322"] + 
    course_taken["L I S 407"] + 
    course_taken["L I S 440"] + 
    course_taken["L I S 464"] + 
    course_taken["L I S 501"] + 
    course_taken["LSC 460"] + 
    course_taken["LSC 660"] + 
    course_taken["MATH 331"] + 
    course_taken["SOC 351"] + 
    course_taken["SOC/C&E SOC 618"] + 
    course_taken["SOC/C&E SOC 693"] + 
    course_taken["SOIL SCI 585"] + 
    course_taken["STAT 405"] + 
    course_taken["STAT 433"] >= 2 - e_ds_machine_learning - e_ds_advanced_computing - e_ds_statistical_modelling
)

#Econ Constraints
@constraint(
    model,
    course_taken["MATH 221"] + 
    (course_taken["MATH 171"] + course_taken["MATH 217"]) + 
    (course_taken["MATH 211"] + course_taken["MATH 213"]) + 
    (course_taken["MATH 211"] + course_taken["ECON 205"]) >= 1
)
@constraint(
    model,
    course_taken["ECON 310"] + 
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["MATH/STAT 309"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["STAT 340"] >= 1
)
@constraint(
    model,
    (course_taken["ECON 101"] + course_taken["ECON 102"]) + 
    course_taken["ECON 111"] + course_taken["ECON 111"]>= 2
)
@constraint(
    model,
    (course_taken["ECON 301"] + course_taken["ECON 302"]) + 
    (course_taken["ECON 311"] + course_taken["ECON 312"]) >= 2
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 409"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 435"] + 
    course_taken["ECON 441"] + 
    course_taken["ECON 442"] + 
    course_taken["ECON 448"] + 
    course_taken["ECON 450"] + 
    course_taken["ECON 451"] + 
    course_taken["ECON 455"] + 
    course_taken["ECON 458"] + 
    course_taken["ECON 460"] + 
    course_taken["ECON 461"] + 
    course_taken["ECON 464"] + 
    course_taken["ECON 467"] + 
    course_taken["ECON 468"] + 
    course_taken["ECON 475"] + 
    course_taken["ECON/FINANCE 503"] + 
    course_taken["ECON 521"] + 
    course_taken["ECON 522"] + 
    course_taken["ECON/RMI 530"] + 
    course_taken["ECON/POP HLTH/PUB AFFR 548"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 580"] + 
    course_taken["ECON 621"] + 
    course_taken["ECON 623"] + 
    course_taken["ECON 661"] + 
    course_taken["ECON 664"] + 
    course_taken["ECON 666"] + 
    course_taken["ECON 690"] + 
    course_taken["ECON 695"] >= 2 + e_econ_two_courses
)
@constraint(
    model,
    course_taken["ECON/FINANCE 300"] + 
    course_taken["ECON/HIST SCI 305"] + 
    course_taken["ECON/AAE/REAL EST/URB R PL 306"] + 
    course_taken["ECON 309"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON/FINANCE 320"] + 
    course_taken["ECON 321"] + 
    course_taken["ECON 330"] + 
    course_taken["ECON/AAE/ENVIR ST 343"] + 
    course_taken["ECON 355"] + 
    course_taken["ECON 370"] + 
    course_taken["ECON/AAE 371"] + 
    course_taken["ECON 390"] + 
    course_taken["ECON/REAL EST/URB R PL 420"] + 
    course_taken["ECON/AAE 421"] + 
    course_taken["ECON/ENVIR ST/POLI SCI/URB R PL 449"] + 
    course_taken["ECON/AAE/INTL BUS 462"] + 
    course_taken["ECON 465"] + 
    course_taken["ECON/HISTORY 466"] + 
    course_taken["ECON/AAE 473"] + 
    course_taken["ECON/AAE 474"] + 
    course_taken["ECON/AAE 477"] + 
    course_taken["ECON/PHILOS 524"] + 
    course_taken["ECON/AAE 526"] + 
    course_taken["ECON/AAE/F&W ECOL 531"] + 
    course_taken["ECON/SOC 663"] + 
    course_taken["ECON/AAE/ENVIR ST/URB R PL 671"] >= 2 - e_econ_two_courses
)

@constraint(
    model,
    sum(credits[course] * course_taken[course] for course in courses) >= 48
)

# Assume you take 18 credits every semester for 8 semesters while considering taking humanities, social sciecnes, language, and natural sciences with a total of 36 credits
@constraint(
    model,
    sum(credits[course] * course_taken[course] for course in courses) <= 108
)

# Maximize the difficulty and number CS courses while making the total number courses minimized
@objective(model, Max, sum(difficulty[course] * course_taken[course] for course in courses if occursin("COMP SCI", course)) - 0.5 * sum(course_taken[course] for course in courses))

set_optimizer_attribute(model, "presolve", "on")
# Solve the optimization problem
optimize!(model)

# Check if a solution was found
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found. Courses to enroll with most CS related:")
    final_taken = []
    for course in courses
        if value(course_taken[course]) > 0.5
            push!(final_taken, course)
            println(course)
        end
    end
else
    println("No optimal solution found.")
end

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 5e+00]
  Cost   [5e-01, 6e+02]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+02]
Presolving model
19 rows, 183 cols, 572 nonzeros  0s
16 rows, 78 cols, 244 nonzeros  0s
11 rows, 66 cols, 151 nonzeros  0s
Objective function is integral with scale 2

Solving MIP model with:
   11 rows
   66 cols (48 binary, 11 integer, 1 implied int., 6 continuous)
   151 nonzeros
MIP-Timing:      0.0074 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | Best

In [23]:
#Print Schedule Result
result = group_courses(final_taken, difficulty)
schedule = create_schedule(result)
for i in 1:8
    semester_key = "Semester $i"
    if length(schedule[semester_key]) > 0
        println("$semester_key: ", join(["$course" for course in schedule[semester_key]], ", "))
    else
        println("$semester_key: No courses scheduled")
    end
end

Semester 1: ("ECON", 111), ("COMP SCI", 200), ("COMP SCI/MATH", 240), ("MATH", 217)
Semester 2: ("COMP SCI", 220), ("MATH", 222), ("COMP SCI/E C E", 252), ("STAT", 240)
Semester 3: ("COMP SCI", 300), ("MATH", 340), ("ECON", 311), ("COMP SCI/E C E", 354)
Semester 4: ("COMP SCI", 320), ("ECON", 312), ("STAT", 340), ("ECON/AAE", 473)
Semester 5: ("COMP SCI", 400), ("ECON", 410), ("L I S", 461), ("COMP SCI/B M I", 576)
Semester 6: ("ECON", 451), ("COMP SCI", 540), ("COMP SCI/DS", 579), ("E C E/I SY E", 570)
Semester 7: ("COMP SCI", 577), ("COMP SCI/BMI", 576), ("ECON/AAE/F&W ECOL", 531), ("COMP SCI/I SY E", 635)
Semester 8: ("COMP SCI", 620), ("COMP SCI", 639), ("COMP SCI", 640), ("COMP SCI", 642)


<!-- 4.2 -->
<a id="sec-4-2"></a>
### 4.2 Case II — Focused Curriculum
[↑ Back to top](#toc-top)

Situation B: Changing Constraints - Looking at the schedule of a student who wants to take the minimum number of courses to triple major, but has come in with a lot of credits and wants to take specific courses. 

This iteration of our model represents a high-acheiving student who came in with both calculus requirements (Math 221 & Math 222) as well as CS200. Additionally, the student is particularly fond of optimization and specifically wants to take CS 524, CS 525, and CS 425. We included this iteration because we assume a student choosing to triple major is an overacheiver so them coming in with ample credits is also very likely. By changing our model to account for the students exisitng work, we increase the realisticness and robustness of our model allowing it to be a useable tool for a wider ranges of cases and students.

To account for these changes constraints were removed (like the CS and Calculus constraints), but the model largely stayed the same. To add another element of different to this case, we added the requriement of optimization classes being taken. Again, we felt this added a dimension of realness to our project and allows students to see that even when taking on a task as difficult as a triple major it's still possible and important to prioritize the classes you're the most interested in. 

In [24]:
using JuMP, GLPK

# Define the model with the GLPK solver
model = Model(GLPK.Optimizer)

# Create a binary variable for each course
@variable(model, course_taken[courses], Bin)

#All elective variables
@variable(model, e_cs_theory >= 0)
@variable(model, e_cs_software_hardware >= 0)
@variable(model, e_cs_applications >= 0)
@variable(model, e_ds_machine_learning >= 0)
@variable(model, e_ds_advanced_computing >= 0)
@variable(model, e_ds_statistical_modelling >= 0)
@variable(model, e_econ_two_courses >= 0)

#Additional Constraints for the Sudent who wants to take some specific courses
@constraint(
    model,
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] +
    course_taken["COMP SCI/E C E/I SY E 524"] +
    course_taken["COMP SCI/I SY E/MATH 425"] >= 3 
)
#Computer Science Constraints
#Remove CS 200 as they already have credits for it
@constraint(
    model,
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI/MATH 240"] + 
    course_taken["COMP SCI/E C E 252"] + 
    course_taken["COMP SCI/E C E 354"] + 
    course_taken["COMP SCI 400"] >= 5
)

@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)

@constraint(
    model,
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["MATH 331"] + 
    course_taken["STAT 333"] + 
    course_taken["STAT 340"] + 
    course_taken["STAT 371"] + 
    course_taken["STAT/MATH 431"] + 
    course_taken["MATH 531"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI 577"] + 
    course_taken["COMP SCI 520"] >= 1 + e_cs_theory
)
@constraint(
    model,
    course_taken["COMP SCI 407"] + 
    course_taken["COMP SCI/E C E 506"] + 
    course_taken["COMP SCI 536"] + 
    course_taken["COMP SCI 538"] + 
    course_taken["COMP SCI 537"] + 
    course_taken["COMP SCI 542"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI/E C E 552"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 640"] + 
    course_taken["COMP SCI 642"] >= 2 + e_cs_software_hardware
)
@constraint(
    model,
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI 534"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["COMP SCI 541"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI 566"] + 
    course_taken["COMP SCI 570"] + 
    course_taken["COMP SCI 571"] >= 1 + e_cs_applications 
)
@constraint(
    model,
    course_taken["COMP SCI/E C E/MATH 435"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH/STAT 475"] + 
    course_taken["COMP SCI/DS/I SY E 518"] + 
    course_taken["COMP SCI/I SY E 526"] + 
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI/E C E/M E 539"] +  
    course_taken["COMP SCI/I SY E/M E 558"] + 
    course_taken["COMP SCI/E C E 561"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["COMP SCI/DS 579"] + 
    course_taken["COMP SCI 620"] + 
    course_taken["COMP SCI/I SY E 635"] + 
    course_taken["COMP SCI 639"] >= 2 - e_cs_theory - e_cs_software_hardware - e_cs_applications
)

#Data Science Constraints
#Remove Calculus constraint
@constraint(
    model,
    course_taken["STAT 240"] +
    course_taken["STAT 340"] +
    course_taken["COMP SCI 220"] + 
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI 320"] +
    course_taken["L I S 461"] + 
    course_taken["E C E/I SY E 570"] >= 7
)
@constraint(
    model,
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E/M E 539"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["GEN BUS 656"] + 
    course_taken["I SY E 521"] + 
    course_taken["MATH 535"] + 
    course_taken["PHYSICS 361"] + 
    course_taken["STAT 451"] + 
    course_taken["STAT 453"] >= 1 + e_ds_machine_learning
)
@constraint(
    model,
    course_taken["COMP SCI 400"] + 
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["GEOG 573"] + 
    course_taken["GEOG 574"] + 
    course_taken["MATH 444"] >= 1 + e_ds_advanced_computing
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 460"] + 
    course_taken["GEOG 579"] + 
    course_taken["I SY E 575"] + 
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT/MATH 310"] + 
    course_taken["STAT 312"] + 
    course_taken["STAT 349"] + 
    course_taken["STAT 351"] + 
    course_taken["STAT 421"] + 
    course_taken["STAT/M E 424"] + 
    course_taken["STAT 436"] + 
    course_taken["STAT 443"] + 
    course_taken["STAT 456"] + 
    course_taken["STAT 461"] + 
    course_taken["STAT 575"] + 
    course_taken["MATH 531"] + 
    course_taken["MATH/I SY E/OTM/STAT 632"] + 
    course_taken["MATH 635"] >= 1 + e_ds_statistical_modelling
)
@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI 577"] + 
    course_taken["E C E 203"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 695"] + 
    course_taken["GEOG 378"] + 
    course_taken["GEOG 572"] + 
    course_taken["GEOG 575"] + 
    course_taken["I SY E 323"] + 
    course_taken["I SY E 412"] + 
    course_taken["I SY E/M E 512"] + 
    course_taken["I SY E 612"] + 
    course_taken["INFO SYS 322"] + 
    course_taken["L I S 407"] + 
    course_taken["L I S 440"] + 
    course_taken["L I S 464"] + 
    course_taken["L I S 501"] + 
    course_taken["LSC 460"] + 
    course_taken["LSC 660"] + 
    course_taken["MATH 331"] + 
    course_taken["SOC 351"] + 
    course_taken["SOC/C&E SOC 618"] + 
    course_taken["SOC/C&E SOC 693"] + 
    course_taken["SOIL SCI 585"] + 
    course_taken["STAT 405"] + 
    course_taken["STAT 433"] >= 2 - e_ds_machine_learning - e_ds_advanced_computing - e_ds_statistical_modelling
)

#Econ Constraints
#Remove Calc constraint in Econ
@constraint(
    model,
    course_taken["ECON 310"] + 
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["MATH/STAT 309"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["STAT 340"] >= 1
)
@constraint(
    model,
    course_taken["ECON 101"] + 
    course_taken["ECON 102"] + 
    2*course_taken["ECON 111"] >= 2
)
@constraint(
    model,
    course_taken["ECON 301"] + course_taken["ECON 302"]  >= 2
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 409"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 435"] + 
    course_taken["ECON 441"] + 
    course_taken["ECON 442"] + 
    course_taken["ECON 448"] + 
    course_taken["ECON 450"] + 
    course_taken["ECON 451"] + 
    course_taken["ECON 455"] + 
    course_taken["ECON 458"] + 
    course_taken["ECON 460"] + 
    course_taken["ECON 461"] + 
    course_taken["ECON 464"] + 
    course_taken["ECON 467"] + 
    course_taken["ECON 468"] + 
    course_taken["ECON 475"] + 
    course_taken["ECON/FINANCE 503"] + 
    course_taken["ECON 521"] + 
    course_taken["ECON 522"] + 
    course_taken["ECON/RMI 530"] + 
    course_taken["ECON/POP HLTH/PUB AFFR 548"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 580"] + 
    course_taken["ECON 621"] + 
    course_taken["ECON 623"] + 
    course_taken["ECON 661"] + 
    course_taken["ECON 664"] + 
    course_taken["ECON 666"] + 
    course_taken["ECON 690"] + 
    course_taken["ECON 695"] >= 2 + e_econ_two_courses
)
@constraint(
    model,
    course_taken["ECON/FINANCE 300"] + 
    course_taken["ECON/HIST SCI 305"] + 
    course_taken["ECON/AAE/REAL EST/URB R PL 306"] + 
    course_taken["ECON 309"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON/FINANCE 320"] + 
    course_taken["ECON 321"] + 
    course_taken["ECON 330"] + 
    course_taken["ECON/AAE/ENVIR ST 343"] + 
    course_taken["ECON 355"] + 
    course_taken["ECON 370"] + 
    course_taken["ECON/AAE 371"] + 
    course_taken["ECON 390"] + 
    course_taken["ECON/REAL EST/URB R PL 420"] + 
    course_taken["ECON/AAE 421"] + 
    course_taken["ECON/ENVIR ST/POLI SCI/URB R PL 449"] + 
    course_taken["ECON/AAE/INTL BUS 462"] + 
    course_taken["ECON 465"] + 
    course_taken["ECON/HISTORY 466"] + 
    course_taken["ECON/AAE 473"] + 
    course_taken["ECON/AAE 474"] + 
    course_taken["ECON/AAE 477"] + 
    course_taken["ECON/PHILOS 524"] + 
    course_taken["ECON/AAE 526"] + 
    course_taken["ECON/AAE/F&W ECOL 531"] + 
    course_taken["ECON/SOC 663"] + 
    course_taken["ECON/AAE/ENVIR ST/URB R PL 671"] >= 2 - e_econ_two_courses
)

@constraint(
    model,
    sum(credits[course] * course_taken[course] for course in courses) >= 48
)

@objective(model, Min, sum(course_taken[course] for course in courses))

# Solve the optimization problem
optimize!(model)

# Check if a solution was found
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found. Courses to enroll with lowest number of courses:")
    final_taken = []
    for course in courses
        if value(course_taken[course]) > 0.5
            push!(final_taken, course)
            println(course)
        end
    end
else
    println("No optimal solution found.")
end

Optimal solution found. Courses to enroll with lowest number of courses:
ECON 400
COMP SCI/E C E 354
COMP SCI/MATH 240
COMP SCI 537
ECON 570
ECON 301
COMP SCI/E C E 252
ECON 321
COMP SCI/E C E/M E 532
COMP SCI 538
STAT 240
ECON 111
ECON 315
ECON 302
COMP SCI 520
COMP SCI 220
COMP SCI/I SY E/MATH/STAT 525
E C E/I SY E 570
COMP SCI/E C E/I SY E 524
MATH 375
COMP SCI 320
COMP SCI 300
L I S 461
COMP SCI/I SY E/MATH 425
COMP SCI 400
STAT 340


In [25]:
#Print Schedule Result
result = group_courses(final_taken, difficulty)
schedule = create_schedule(result)
for i in 1:8
    semester_key = "Semester $i"
    if length(schedule[semester_key]) > 0
        println("$semester_key: ", join(["$course" for course in schedule[semester_key]], ", "))
    else
        println("$semester_key: No courses scheduled")
    end
end

Semester 1: ("ECON", 111), ("COMP SCI", 220), ("COMP SCI/MATH", 240), ("COMP SCI/E C E", 252)
Semester 2: ("STAT", 240), ("COMP SCI", 300), ("MATH", 375), ("ECON", 301)
Semester 3: ("COMP SCI", 320), ("ECON", 302), ("COMP SCI/E C E", 354), ("STAT", 340)
Semester 4: ("ECON", 315), ("COMP SCI/I SY E/MATH", 425), ("COMP SCI", 400), ("L I S", 461)
Semester 5: ("ECON", 321), ("COMP SCI", 520), ("COMP SCI/E C E/M E", 532), ("COMP SCI/E C E/I SY E", 524)
Semester 6: ("ECON", 400), ("COMP SCI", 537), ("E C E/I SY E", 570), ("COMP SCI/I SY E/MATH/STAT", 525)
Semester 7: ("COMP SCI", 538), ("ECON", 570)
Semester 8: No courses scheduled


<!-- 4.3 -->
<a id="sec-4-3"></a>
### 4.3 Case III 
[↑ Back to top](#toc-top)

Situation C: Scheduling Optimization - Looking at the schedule of a student who wants to take the minimum number of courses to triple major, but adds their third major late, specifcially in the 4th semester (second-year) of university.

The final iteration of our course project simulates a student’s decision to triple major. The student starts with a double major in Data Science and Computer Science and later adds Economics as a third major. Due to the late addition, no Economics courses are taken before the switch. This reflects a common scenario where students, unsure of their academic options during their first year, may decide to pursue an additional major later in their studies.

Our model aims to help students understand that adding a major in their second or third year is possible without delaying graduation. It provides students with the tools and confidence to make informed decisions about their academic paths, demonstrating that they can still succeed and graduate on time, even when making risky decisions.

In [26]:
#We have modified our create_schedule function to account for the fact that the student picks up the new Economics major in the fourth semester. 
#This is to ensure that we can allocate all the courses correctly and make a schedule according to the opitmal solution of courses they are required to take.

function create_schedule_major(courses::Dict{String, Vector{Int}}, major_name::String, start::Int)
    # Check if the starting semester is not ideal (5 or later)
    if start >= 5
        return
    end
    
    # Initialize structures
    sorted_courses = []
    schedule = Dict("Semester $i" => [] for i in 1:8)
    unadded_courses = []

    # Flatten the courses dictionary into a list of (subject, course) tuples
    for (subject, course_list) in courses
        append!(sorted_courses, [(subject, course) for course in course_list])
    end

    # Sort the courses by group (major vs non-major) and by course number within each group
    sorted_courses = sort(sorted_courses, by = x -> (x[1] != major_name, parse(Int, string(x[2])[1])))

    # Schedule courses, first non-major courses, then major courses from the specified start semester
    for (subject, course) in sorted_courses
        added = false

        # Determine the starting semester for scheduling based on whether it's the specified major
        semester_start = subject == major_name ? start : 1

        for semester in semester_start:8
            if length(schedule["Semester $semester"]) < 4 && !(subject in [x[1] for x in schedule["Semester $semester"]])
                push!(schedule["Semester $semester"], (subject, course))
                added = true
                break
            end
        end

        if !added
            push!(unadded_courses, (subject, course))
        end
    end

    # Attempt to place unadded courses in any available slots, respecting the major start constraint
    for (subject, course) in unadded_courses
        for semester in (subject == major_name ? start : 1):8
            if length(schedule["Semester $semester"]) < 4
                push!(schedule["Semester $semester"], (subject, course))
                break
            end
        end
    end

    return schedule
end




create_schedule_major (generic function with 1 method)

In [27]:
using JuMP, GLPK

# Define the model with the GLPK solver
model = Model(GLPK.Optimizer)

# Create a binary variable for each course
@variable(model, course_taken[courses], Bin)

#All elective variables
@variable(model, e_cs_theory >= 0)
@variable(model, e_cs_software_hardware >= 0)
@variable(model, e_cs_applications >= 0)
@variable(model, e_ds_machine_learning >= 0)
@variable(model, e_ds_advanced_computing >= 0)
@variable(model, e_ds_statistical_modelling >= 0)
@variable(model, e_econ_two_courses >= 0)

#Computer Science Constraints
@constraint(
    model,
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI/MATH 240"] + 
    course_taken["COMP SCI/E C E 252"] + 
    course_taken["COMP SCI/E C E 354"] + 
    course_taken["COMP SCI 200"] + 
    course_taken["COMP SCI 400"] >= 6 
)

@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)

@constraint(
    model,
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["MATH 331"] + 
    course_taken["STAT 333"] + 
    course_taken["STAT 340"] + 
    course_taken["STAT 371"] + 
    course_taken["STAT/MATH 431"] + 
    course_taken["MATH 531"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI 577"] + 
    course_taken["COMP SCI 520"] >= 1 + e_cs_theory
)
@constraint(
    model,
    course_taken["COMP SCI 407"] + 
    course_taken["COMP SCI/E C E 506"] + 
    course_taken["COMP SCI 536"] + 
    course_taken["COMP SCI 538"] + 
    course_taken["COMP SCI 537"] + 
    course_taken["COMP SCI 542"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI/E C E 552"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 640"] + 
    course_taken["COMP SCI 642"] >= 2 + e_cs_software_hardware
)
@constraint(
    model,
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI 534"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["COMP SCI 541"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI 566"] + 
    course_taken["COMP SCI 570"] + 
    course_taken["COMP SCI 571"] >= 1 + e_cs_applications
)
@constraint(
    model,
    course_taken["COMP SCI/E C E/MATH 435"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH/STAT 475"] + 
    course_taken["COMP SCI/DS/I SY E 518"] + 
    course_taken["COMP SCI/I SY E 526"] + 
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI/E C E/M E 539"] +  
    course_taken["COMP SCI/I SY E/M E 558"] + 
    course_taken["COMP SCI/E C E 561"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["COMP SCI/DS 579"] + 
    course_taken["COMP SCI 620"] + 
    course_taken["COMP SCI/I SY E 635"] + 
    course_taken["COMP SCI 639"] >= 2 - e_cs_theory - e_cs_software_hardware - e_cs_applications
)

#Data Science Constraints
@constraint(
    model,
    course_taken["MATH 221"] + course_taken["MATH 217"] >= 1
)

@constraint(
    model,
    course_taken["MATH 222"] >= 1
)
@constraint(
    model,
    course_taken["STAT 240"] +
    course_taken["STAT 340"] +
    course_taken["COMP SCI 220"] + 
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI 320"] +
    course_taken["L I S 461"] + 
    course_taken["E C E/I SY E 570"] >= 7
)
@constraint(
    model,
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E/M E 539"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["GEN BUS 656"] + 
    course_taken["I SY E 521"] + 
    course_taken["MATH 535"] + 
    course_taken["PHYSICS 361"] + 
    course_taken["STAT 451"] + 
    course_taken["STAT 453"] >= 1 + e_ds_machine_learning
)
@constraint(
    model,
    course_taken["COMP SCI 400"] + 
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["GEOG 573"] + 
    course_taken["GEOG 574"] + 
    course_taken["MATH 444"] >= 1 + e_ds_advanced_computing
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 460"] + 
    course_taken["GEOG 579"] + 
    course_taken["I SY E 575"] + 
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT/MATH 310"] + 
    course_taken["STAT 312"] + 
    course_taken["STAT 349"] + 
    course_taken["STAT 351"] + 
    course_taken["STAT 421"] + 
    course_taken["STAT/M E 424"] + 
    course_taken["STAT 436"] + 
    course_taken["STAT 443"] + 
    course_taken["STAT 456"] + 
    course_taken["STAT 461"] + 
    course_taken["STAT 575"] + 
    course_taken["MATH 531"] + 
    course_taken["MATH/I SY E/OTM/STAT 632"] + 
    course_taken["MATH 635"] >= 1 + e_ds_statistical_modelling
)
@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI 577"] + 
    course_taken["E C E 203"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 695"] + 
    course_taken["GEOG 378"] + 
    course_taken["GEOG 572"] + 
    course_taken["GEOG 575"] + 
    course_taken["I SY E 323"] + 
    course_taken["I SY E 412"] + 
    course_taken["I SY E/M E 512"] + 
    course_taken["I SY E 612"] + 
    course_taken["INFO SYS 322"] + 
    course_taken["L I S 407"] + 
    course_taken["L I S 440"] + 
    course_taken["L I S 464"] + 
    course_taken["L I S 501"] + 
    course_taken["LSC 460"] + 
    course_taken["LSC 660"] + 
    course_taken["MATH 331"] + 
    course_taken["SOC 351"] + 
    course_taken["SOC/C&E SOC 618"] + 
    course_taken["SOC/C&E SOC 693"] + 
    course_taken["SOIL SCI 585"] + 
    course_taken["STAT 405"] + 
    course_taken["STAT 433"] >= 2 - e_ds_machine_learning - e_ds_advanced_computing - e_ds_statistical_modelling
)

#Econ Constraints
@constraint(
    model,
    2*course_taken["MATH 221"] + 
    course_taken["MATH 211"] + 
    course_taken["ECON 205"] >= 2
)
@constraint(
    model,
    course_taken["ECON 310"] + 
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["MATH/STAT 309"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["STAT 340"] >= 1
)
@constraint(
    model,
    course_taken["ECON 101"] + 
    course_taken["ECON 102"] + 
    2*course_taken["ECON 111"] >= 2
)
@constraint(
    model,
    course_taken["ECON 301"] + course_taken["ECON 302"]  >= 2
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 409"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 435"] + 
    course_taken["ECON 441"] + 
    course_taken["ECON 442"] + 
    course_taken["ECON 448"] + 
    course_taken["ECON 450"] + 
    course_taken["ECON 451"] + 
    course_taken["ECON 455"] + 
    course_taken["ECON 458"] + 
    course_taken["ECON 460"] + 
    course_taken["ECON 461"] + 
    course_taken["ECON 464"] + 
    course_taken["ECON 467"] + 
    course_taken["ECON 468"] + 
    course_taken["ECON 475"] + 
    course_taken["ECON/FINANCE 503"] + 
    course_taken["ECON 521"] + 
    course_taken["ECON 522"] + 
    course_taken["ECON/RMI 530"] + 
    course_taken["ECON/POP HLTH/PUB AFFR 548"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 580"] + 
    course_taken["ECON 621"] + 
    course_taken["ECON 623"] + 
    course_taken["ECON 661"] + 
    course_taken["ECON 664"] + 
    course_taken["ECON 666"] + 
    course_taken["ECON 690"] + 
    course_taken["ECON 695"] >= 2 + e_econ_two_courses
)
@constraint(
    model,
    course_taken["ECON/FINANCE 300"] + 
    course_taken["ECON/HIST SCI 305"] + 
    course_taken["ECON/AAE/REAL EST/URB R PL 306"] + 
    course_taken["ECON 309"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON/FINANCE 320"] + 
    course_taken["ECON 321"] + 
    course_taken["ECON 330"] + 
    course_taken["ECON/AAE/ENVIR ST 343"] + 
    course_taken["ECON 355"] + 
    course_taken["ECON 370"] + 
    course_taken["ECON/AAE 371"] + 
    course_taken["ECON 390"] + 
    course_taken["ECON/REAL EST/URB R PL 420"] + 
    course_taken["ECON/AAE 421"] + 
    course_taken["ECON/ENVIR ST/POLI SCI/URB R PL 449"] + 
    course_taken["ECON/AAE/INTL BUS 462"] + 
    course_taken["ECON 465"] + 
    course_taken["ECON/HISTORY 466"] + 
    course_taken["ECON/AAE 473"] + 
    course_taken["ECON/AAE 474"] + 
    course_taken["ECON/AAE 477"] + 
    course_taken["ECON/PHILOS 524"] + 
    course_taken["ECON/AAE 526"] + 
    course_taken["ECON/AAE/F&W ECOL 531"] + 
    course_taken["ECON/SOC 663"] + 
    course_taken["ECON/AAE/ENVIR ST/URB R PL 671"] >= 2 - e_econ_two_courses
)

@constraint(
    model,
    sum(credits[course] * course_taken[course] for course in courses) >= 48
)

@objective(model, Min, sum(difficulty[course] * course_taken[course] for course in courses if occursin("ECON", course)))

# Solve the optimization problem
optimize!(model)

# Check if a solution was found
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found. Courses to enroll with lowest number of courses:")
    final_taken = []
    for course in courses
        if value(course_taken[course]) > 0.5
            push!(final_taken, course)
            println(course)
        end
    end
else
    println("No optimal solution found.")
end

Optimal solution found. Courses to enroll with lowest number of courses:
ECON 400
STAT 433
COMP SCI/E C E 354
COMP SCI/MATH 240
SOIL SCI 585
ECON 409
COMP SCI 537
ECON 301
COMP SCI 412
COMP SCI/E C E 252
COMP SCI/DS/I SY E 518
COMP SCI/E C E/M E 532
ECON/HIST SCI 305
STAT 240
MATH 222
ECON 111
ECON 302
COMP SCI 520
COMP SCI 220
E C E/I SY E 570
MATH 375
COMP SCI 564
COMP SCI 200
COMP SCI 320
COMP SCI 300
L I S 461
COMP SCI 400
MATH 221
ECON/FINANCE 300
STAT 340


In [28]:
#Print Schedule Result
result = group_courses(final_taken, difficulty)
# It is not ideal to add one more major after semester 4
schedule = create_schedule_major(result, "ECON", 4)
if schedule != nothing
    for i in 1:8
        semester_key = "Semester $i"
        if length(schedule[semester_key]) > 0
            println("$semester_key: ", join(["$course" for course in schedule[semester_key]], ", "))
        else
            println("$semester_key: No courses scheduled")
        end
    end
end

if schedule == nothing
    println("It is not ideal to take a major after semester four!")
end

Semester 1: ("COMP SCI", 200), ("COMP SCI/MATH", 240), ("MATH", 221), ("COMP SCI/E C E", 252)
Semester 2: ("COMP SCI", 220), ("MATH", 222), ("STAT", 240), ("ECON/FINANCE", 300)
Semester 3: ("COMP SCI", 300), ("MATH", 375), ("ECON/HIST SCI", 305), ("COMP SCI/E C E", 354)
Semester 4: ("ECON", 111), ("COMP SCI", 320), ("STAT", 340), ("L I S", 461)
Semester 5: ("ECON", 301), ("COMP SCI", 400), ("STAT", 433), ("SOIL SCI", 585)
Semester 6: ("ECON", 302), ("COMP SCI", 412), ("COMP SCI/E C E/M E", 532), ("E C E/I SY E", 570)
Semester 7: ("ECON", 400), ("COMP SCI", 520), ("COMP SCI/DS/I SY E", 518), ("COMP SCI", 564)
Semester 8: ("ECON", 409), ("COMP SCI", 537)


All of our models were sucessfully optimized with slightly different overall solutions/schedules. Henceforth, our model shows that a triple major at UW is possible if the right courses are taken.  Initially, the model focuses on meeting a broad set of multi-domain requirements with the smallest possible number of courses, demonstrating a baseline efficiency valuable for any student trying to graduate promptly. As we progress, the scenarios introduce increasingly nuanced priorities: one emphasizes the strategic selection of high-difficulty CS courses to align with a student’s specialized interests; another incorporates specific, advanced methodological courses, reflecting personal academic goals beyond standard requirements. The model also proves capable of adjusting to more realistic contexts, such as entering university with pre-earned credits or deciding to add a new major after the initial semesters. In each case, the solver adapts, finding feasible pathways that meet all domain constraints—sometimes even under altered objectives, like minimizing the difficulty of chosen Economics courses or ensuring certain majors are added late without delaying graduation.

We believe a tool like this should be widely available and used for all students, advisors, and faculty to allow for ease of scheduling. Our model is an exciting example of how technology and computers can make tasks significantly more efficient than if a human performend it. For fun, we tried to manually map out a CS and Econ double major, a tasks that hundreds of UW Students do every single year. It took nearly an hour and 80+ clicks back and forth between major pages. Our model, took this problem with an additional major and did it in a matter of seconds, showing the models use and effectiveness. 

<!-- 5 -->
<a id="sec-5"></a>
## 5. Limitations, Future Work, and Conclusion
[↑ Back to top](#toc-top)

Limitations:

It is worth noting that this model only takes core courses into account when creating a schedule for a student. In reality, a student has to also take a lot of general education and breadth courses such as humantiies, social science, and literature credits. This most defintley complicates the model, considering there are so many combinations and possibilities of the same. Moreover, our model is only limited to three majors, however, we can easily scale this to as many majors we want by adding the correct data and constraints. This would involve adding a lot more constraints to the model as each requirement for the major would be a constraint. Finally, our schedule assumes that a student can take 4 core ourses a semester. While it is possible to do so and students do in fact do this, this can be quite rigorous and can not be done by everyone, hence, making it hard to triple major, yet feasible to do so.

Future Work:

Furthermore, this is defintley a very good base for future work as optimizing schedules at UW-Madison can be an amazing tool for students to use. If we can have a methodlogy to input any major at UW-Madison and create a course schedule for the same, then it can be applicable to all UW-Madison students who want to pick up more than one major. This is a great project that can be built in the future, a tool for all students to use along with their DARS (Degree Audit Reporting System) reports that they regularly do. One can also attempt to major in between departments (for example, let us say engineering school and the letters and science school). This would complicate the model a lot more, but certainly can be implimented.


Conclusion:

Through this project, we successfully developed an optimization model that identifies the most efficient class schedule for students pursuing a triple major in Data Science, Economics, and Computer Science. By evaluating a range of objective functions, we were able to determine the optimal schedule that minimizes time to graduation while meeting all course requirements and student preferences. These objective functions considered factors such as course prerequisites, credit limits, and the overlap of courses across the three majors, ensuring that students can meet their academic goals without unnecessary delays or excess courses.

This approach not only provides a clear roadmap for students seeking a triple major, but also offers flexibility for future adjustments. In particular, the model can be extended to support other combinations of majors and minors. One potential direction is to build an input function that allows students to specify which majors they are interested in, enabling the model to generate personalized schedules for any combination of fields. This could be especially helpful for students who want to explore interdisciplinary studies beyond the three majors we considered. A tool at this scale would be all the more valuable to advisors and prospective students to see the true potential for opportunity here at UW-Madison.

Overall, this project not only highlights the potential of optimization techniques to solve complex scheduling problems, but also sets the stage for expanding the model to serve a broader range of students and academic pursuits. With further development, this tool could become an essential resource for students navigating the demands of interdisciplinary education.






